In [3]:
from src.load_data import get_data
from src.EchoGAE import EchoGAE_algorithm
from src.echo_chamber_measure import EchoChamberMeasure


# TODO: Work on baselines
import numpy as np
from networkx.algorithms.community import asyn_fluidc
from src.baselines.RWC_jit import RWC
# from src.baselines.RWC import RWC # If you don't have numba installed use this line instead but it will be slower

from src.baselines.polarization_index import add_ideology_to_graph, opinion_model, get_polarization_index

/Users/faisalalatawi/Documents/GitHub/echo-chamber-score/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
datasets = ["gun", "abortion", "super_bowl", "sxsw"]
ds = datasets[3]

# Echo Chamber Score:

In [5]:
G, users_embeddings, labels, allsides_scores, node_id_map = get_data(f"data/{ds}/")

In [6]:
user_emb = EchoGAE_algorithm(G, user_embeddings= users_embeddings,show_progress=True, hidden_channels = 20, out_channels=10, epochs=300)

Epoch: 001, AUC: 0.7830, AP: 0.8047
Epoch: 002, AUC: 0.8023, AP: 0.8171
Epoch: 003, AUC: 0.8070, AP: 0.8196
Epoch: 004, AUC: 0.7957, AP: 0.8129
Epoch: 005, AUC: 0.7762, AP: 0.8016
Epoch: 006, AUC: 0.7538, AP: 0.7874
Epoch: 007, AUC: 0.7386, AP: 0.7791
Epoch: 008, AUC: 0.7307, AP: 0.7745
Epoch: 009, AUC: 0.7271, AP: 0.7724
Epoch: 010, AUC: 0.7270, AP: 0.7726
Epoch: 011, AUC: 0.7283, AP: 0.7738
Epoch: 012, AUC: 0.7309, AP: 0.7757
Epoch: 013, AUC: 0.7363, AP: 0.7797
Epoch: 014, AUC: 0.7514, AP: 0.7889
Epoch: 015, AUC: 0.7818, AP: 0.8062
Epoch: 016, AUC: 0.8247, AP: 0.8319
Epoch: 017, AUC: 0.8599, AP: 0.8575
Epoch: 018, AUC: 0.8782, AP: 0.8729
Epoch: 019, AUC: 0.8853, AP: 0.8805
Epoch: 020, AUC: 0.8881, AP: 0.8837
Epoch: 021, AUC: 0.8879, AP: 0.8839
Epoch: 022, AUC: 0.8857, AP: 0.8815
Epoch: 023, AUC: 0.8819, AP: 0.8766
Epoch: 024, AUC: 0.8766, AP: 0.8691
Epoch: 025, AUC: 0.8712, AP: 0.8617
Epoch: 026, AUC: 0.8659, AP: 0.8565
Epoch: 027, AUC: 0.8636, AP: 0.8555
Epoch: 028, AUC: 0.8637, AP:

In [7]:
ecm = EchoChamberMeasure(user_emb, labels)

In [8]:
ecm.echo_chamber_index()

0.45679116523536495

# Baselines:

## 1. Random Walk Controversy Score

In [9]:
mem = asyn_fluidc(G, k=2, seed=42, max_iter=1000)
mem = list(mem)

nodes_0 = np.array(list(mem[0]))
nodes_1 = np.array(list(mem[1]))

In [10]:
RWC(G, nodes_0, nodes_1)

0.480910948932565

## 2. Polarization Index:

In [11]:
def add_ideology_to_graph_22(G, ideologies_dict):
    G = G.copy()
    core_nodes = []
    for node in G.nodes():
        if node in ideologies_dict:
            G.nodes[node]["ideology"] = ideologies_dict[node]
            core_nodes.append(node)
        else:
            G.nodes[node]["ideology"] = 0
    return G, core_nodes

In [12]:
G_ven, core_nodes = add_ideology_to_graph_22(G, allsides_scores)
opinions = opinion_model(G_ven, core_nodes=core_nodes)
polarization_index = get_polarization_index(opinions)[0]

In [13]:
polarization_index

0.0015141484435409224